#### Importing Necessary Libraries

In [1]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt
%matplotlib inline

#### Getting the Data In

In [2]:
data_file_of_last_slice_of_CT_scan = pydicom.dcmread('data/P1152/P1152_pCT_20140226_136.dcm') 
pixel_array = data_file_of_last_slice_of_CT_scan.pixel_array #pixel array of ct scan

May want to also try `read_file` function of pydicom (instead of `dcmread` later to see if that makes finding contours easier.

#### Exploring the Data Type and Shape

This is a **numpy array** of values that indicates the brightness of each pixel. This means all numpy array functions and numpy-compatible libraries (such as sk-image) can be used to analyze.

In [3]:
type(pixel_array)

numpy.ndarray

In [4]:
pixel_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [5]:
pixel_array.shape

(512, 512)

#### Exploring the Values of Pixels in the Array

In [6]:
print('The average value of a pixel in the array is {0:.0f}'.format(pixel_array.mean()))
print('The standard deviation of pixel values in the array is {0:.0f}'.format(pixel_array.std()))
print('The minimum value of a pixel in the array {}'.format(pixel_array.min()))
print('The maximum value of a pixel in the array {}'.format(pixel_array.max()))

The average value of a pixel in the array is 171
The standard deviation of pixel values in the array is 343
The minimum value of a pixel in the array 0
The maximum value of a pixel in the array 2607
